<a href="https://colab.research.google.com/github/HayeonLee/Meta_Learning_Tutorial/blob/master/Meta_Learning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Code: https://github.com/abdulfatir/prototypical-networks-tensorflow

# Few-shot Learning 1: Prototypical Network

이 튜토리얼에서 우리는 meta knowledge를 학습하여 few-shot learning을 수행하는 두가지 대표적인 네트워크, [Prototypical Network](https://arxiv.org/abs/1703.05175) (Prototypical Network)과 [Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks](https://arxiv.org/abs/1703.03400) (MAML)을 배울 것입니다.

Prototypcial Network (2 hour)

https://github.com/HayeonLee/Meta_Learning_Tutorial


Model Agnostic Meta Learning (2hour)

https://github.com/haebeom-lee/maml_skhynix




## Meta-learning이란?

Few-shot Classification: 데이터가 많은 기존의 Classification문제와는 다르게 각 클래스별로 참고 할 수 있는 레퍼런스 데이터(shot)가 몇 개(few)밖에 없는 경우.

Meta-learning: few-shot classification을 풀기 위한 meta-knowledge학습.

Epsiode 훈련 방식 이용.

Prototypical Net --> 메트릭 공간에 임베딩

MAML --> 좋은 초기 파라메터 값과 태스크가 주어졌을때 유연하게 파라메터를 업데이트.
![protonet](https://drive.google.com/uc?id=1_zYrjsvPnkOM-S_EBce6eSvr_13fGuul)

## Prototypical Network란?
Few-shot classification을 잘 수행 할 수 있는 메트릭 공간 (metric space)를 배웁니다.


구체적으로, 메트릭 공간에서 각 클래스를 대표 할 수 있는 프로토타입과 주어진 쿼리를 임베딩합니다.

각 프로토타입과 주어진 쿼리 간의 거리를 측정하고, 가장 가까운 거리에 해당하는 프로토타입의 클래스에 쿼리를 할당하는 클러스터링 문제라고 볼 수 있습니다.

![protonet](https://drive.google.com/uc?id=1mdv1z3BlFzJm3SaAWW2hajs9eqoXU5qe)

## Prototypcial Network 튜토리얼

이제부터 어떻게 네트워크를 만들고 Prototypical Network 학습 시킬지를 배워봅시다.
순서는 다음과 같습니다.
0. 데이터셋을 다운로드 받는다.
1. Tensorflow와 다른 library들을 불러온다.
2. 데이터셋을 전처리한다.
3. 모델을 만든다.
4. loss와 optimizer를 정의한다.
5. Training loop를 정의한다.
6. Training!
7. Test

그럼 하나씩 진행해보도록 합시다.



### Download Omniglot Dataset


In [0]:
!mkdir -p data/omniglot/data
!mkdir -p data/omniglot/split
!wget -O data/omniglot/split/trainval.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/trainval.txt?raw=true
!wget -O data/omniglot/split/train.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/train.txt?raw=true
!wget -O data/omniglot/split/test.txt https://github.com/kvpratama/prototypical-networks-tensorflow/blob/master/data/omniglot/splits/test.txt?raw=true
!wget -O images_background.zip https://github.com/brendenlake/omniglot/blob/master/python/images_background.zip?raw=true
!wget -O images_evaluation.zip https://github.com/brendenlake/omniglot/blob/master/python/images_evaluation.zip?raw=true
!unzip images_background.zip -d data/omniglot/data
!unzip images_evaluation.zip -d data/omniglot/data
!mv data/omniglot/data/images_background/* data/omniglot/data/
!mv data/omniglot/data/images_evaluation/* data/omniglot/data/
!rm images_background.zip
!rm images_evaluation.zip

### Import Tensorflow and other libraries


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
#기존에 설치된 다른 버전의 tensorflow를 제거합니다.
!pip uninstall tensorboard -y
!pip uninstall tensorflow-gpu -y
!pip uninstall tensorflow -y

In [0]:
!pip install tensorflow-gpu==1.14.0 #tensorflow gpu 버전을 설치합니다

In [0]:
import tensorflow as tf # tensorflow를 import해줍니다
tf.__version__ # 내가 사용할 tensorflow의 버전을 나타냅니다

In [0]:
# 필요한 라이브러리를 import합니다.
%matplotlib inline
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

### Prepare the dataset

[Omniglot 데이터셋](https://github.com/brendenlake/omniglot) 을 이용하여 Prototypical Network를 학습시켜봅시다.
![Omniglot](https://drive.google.com/uc?id=1XGSSxkHUAk6JM6vli6rCd4CJpi1WsqLZ)
Omniglot 데이터 셋은 50개의 다른 문자(e.g. 한글, 한자)에서 수집한 1623개 글자(e.g. ㅃ, ㅚ)에 대해 손으로 쓰여진 이미지로 구성되어있습니다. 

글자마다 다른 사람들이 쓴 20개의 예제 이미지가 존재합니다.

각 글자를 클래스로 간주하여, 우리는 주어진 글자 이미지를 분류하는 문제를 풀 것입니다.

1200개는 메타 훈련 데이터 셋, 423개는 메타 테스트 데이터 셋이고, 90도 회전하는 Data augmentation을 통해 총 1200 * 4 = 4800개의 훈련용 클래스가 존재합니다.

각 글자 이미지는 흑백 이미지이며, 크기는 28 * 28입니다.

학습이 끝나면 Prototypical Network은 처음 보는 Task들에 대해서 few-shot classification을 잘 수행하게 됩니다.



주어진 정보들을 이용하여 빈 칸을 채워보세요! 

([    ]가 빈 칸을 나타냅니다. 괄호를 지우고 알맞은 코드를 써주세요. 각 0.5점 - 총 2점)

1. 매 epoch마다 100가지의 다른 에피소드가 주어집니다.
2. 매 에피소드마다 태스크의 분류해야할 클래스의 종류는 5개입니다.
3. 태스크의 각 클래스 별로 참고 할 수 있도록 주어진 이미지는 5개입니다.
4. 태스크의 각 클래스 별로 맞춰야할 쿼리 이미지는 15개입니다.


In [0]:
tf.reset_default_graph() 
n_epochs = 20 # epoch 수
n_episodes = [ ] # 1번
n_way = [ ] # 2번
n_shot = [ ] # 3번
n_query = [ ] # 4번
n_examples = 20
im_width = 28 # 이미지 넓이
im_height = 28 # 이미지 높이
channels = 1 # 흑백 이미지 - 단일 채널

In [0]:
# 훈련 데이터 셋을 불러옵니다.
root_dir = './data/omniglot' 
train_split_path = os.path.join(root_dir, 'split', 'trainval.txt') # 경로: ./data/omniglot/split/trainval.txt

# 모든 클래스의 경로를 train_classes 리스트에 저장합니다.
with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()] 
n_classes = len(train_classes)

# 각 클래스별 모든 이미지의 경로를 불러옵니다.
train_dataset = np.zeros([n_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(train_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    # 이미지를 불러들여 회전, 크기 변환, 형 변환 등을 수행한 뒤 train_dataset 행렬에 저장합니다.
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        train_dataset[i, j] = im
c, ni, w, h = train_dataset.shape
print('불러온 훈련 데이터 셋의 특징')
print('클래스 갯수: {}, 클래스 별 이미지 갯수: {}, 이미지 넓이: {}, 이미지 높이: {}'.format(c, ni, w, h))

## Create the models
이제 Prototypical Network를 만들어봅시다!

우리가 만들 Prototypical Network의 구조는 아래 그림들과 같습니다.
![대체 텍스트](https://drive.google.com/uc?id=1nBP6VvWyDLb_kGmg3a_oEk4FP0oJSiXa)

빈 칸을 채워서 그림과 맞는 모델을 만들어보세요 (각 0.5점 - 총 1점). 

In [0]:
h_dim = [ ] # 1번 hidden channels
z_dim = 64 # output channels
kernel_size = [ ] # 2번 kernel size

In [0]:
def conv_block(inputs, out_channels, name='conv'):
    with tf.variable_scope(name):
        # Conv
        conv = tf.layers.conv2d(inputs, out_channels, kernel_size=kernel_size, padding='SAME')
        # BN
        conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True) 
        # ReLU
        conv = tf.nn.relu(conv)
        # Max Pool
        conv = tf.contrib.layers.max_pool2d(conv, 2)
        return conv

In [0]:
def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope('encoder', reuse=reuse):
        net = conv_block(x, h_dim, name='Conv_Block_1')
        net = conv_block(net, h_dim, name='Conv_Block_2')
        net = conv_block(net, h_dim, name='Conv_Block_3')
        net = conv_block(net, z_dim, name='Conv_Block_4')
        #net = keras.layers.flatten(net)
        net = tf.contrib.layers.flatten(net)
        return net

## Define metric function: Euclidean distance
유클리디안 거리의 정의는 다음과 같습니다.
\begin{equation*}
d(z, z') = ||z-z'||^2
\end{equation*}
\begin{equation*}
where\ z, z' \in \mathbb{R}^D
\end{equation*}

Prototypical Network 관점에서 해석하면 임의의 k번째 클래스의 prototype $\mathbf{c}_k$와 주어진 이미지 $\mathbf{x}$가 인코딩된 임베딩 벡터 $f_\phi(\mathbf{x})$ 간의 유클리디안 거리를 구하는데 사용됩니다.
\begin{equation*}
d(f_\phi(\mathbf{x}), \mathbf{c}_k)) = ||f_\phi(\mathbf{x})-\mathbf{c}_k||^2
\end{equation*}



In [0]:
# N개의 임베딩 벡터와 M개의 임베딩 벡터 간의 유클리디안 거리를 계산합니다.
def euclidean_distance(Z, Z_prime):
    N, D = tf.shape(Z)[0], tf.shape(Z)[1] # Z.shape = N x D
    M = tf.shape(Z_prime)[0] # Z_prime.shape = M x D
    Z = tf.tile(tf.expand_dims(Z, axis=1), (1, M, 1)) # Z를 M만큼 복사 - 크기: N x M x D
    Z_prime = tf.tile(tf.expand_dims(Z_prime, axis=0), (N, 1, 1)) # Z'를 N만큼 복사 - 크기: N x M x D
    return tf.reduce_mean(tf.square(Z - Z_prime), axis=2) # ||z-z'||^2 

(참고) euclidean_distance 함수는 편의를 위하여 다음과 같이 구성한 뒤, 한번에 유클리디안 거리를 계산합니다.
\begin{equation*}
Z = [z_1, z_2, ..., z_N]
\end{equation*}
\begin{equation*}
Z' = [z'_1, z'_2, ..., z'_M]
\end{equation*}

## Set placeholders

In [0]:
supports = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
queries = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
support_shape = tf.shape(supports)
query_shape = tf.shape(queries)
num_classes, num_support = support_shape[0], support_shape[1]
num_queries = query_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)

## Get prototypes and query embeddings

![대체 텍스트](https://drive.google.com/uc?id=1mN4CNa9AOq2nQjK8hvXN7KztZHwZoLDE)

위의 그림과 같이 metric space에 클래스의 프로토타입들과 쿼리 이미지들을 임베딩 합니다.

k-th 클래스의 프로토 타입($\mathbb{c}_k$)은 오른쪽 식과 같이 k-th 클래스에 속하는 서포트 이미지들의 임베딩 벡터($f_\phi(\mathbb{x_i}$))의 평균입니다.

예를 들어 10 way 5 shot 문제라면 프로토 타입은 5개의 서포트 임베딩의 평균입니다. 

 쿼리 이미지들은 개별적으로 임베딩됩니다.

In [0]:
# support 이미지를 임베딩 벡터로 인코딩합니다.
emb_supports = encoder(tf.reshape(supports, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
emb_dim = tf.shape(emb_supports)[-1]
# 각 클래스의 support 임베딩들의 평균을 구하여 각 클래스별 프로토타입을 얻습니다.
prototypes = tf.reduce_mean(tf.reshape(emb_supports, [num_classes, num_support, emb_dim]), axis=1)
# 주어진 쿼리를 임베딩 벡터로 인코딩합니다.
emb_queries = encoder(tf.reshape(queries, [num_classes * num_queries, im_height, im_width, channels]), h_dim, z_dim, reuse=True)

## Get prototypical loss with Euclidean distance

![대체 텍스트](https://drive.google.com/uc?id=1LBkqCntsBNiP7RyaCazXopswhslabhJE)


In [0]:
# 쿼리 임베딩들과 프로토타입들 간의 유클리디안 거리를 계산합니다.
dists = euclidean_distance(emb_queries, prototypes)
# log(softmax(x)) 값을 구합니다.
log_p_y = tf.reshape(tf.nn.log_softmax(-dists), [num_classes, num_queries, -1])
# target class의 확률에 -를 붙여 minimize할 metric loss를 구합니다.  
ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
# 정확도를 계산합니다.
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(log_p_y, axis=-1), y), dtype=tf.float32))
# adam optimizer로 loss를 minimize합니다.
train_op = tf.train.AdamOptimizer().minimize(ce_loss)

## Set session

In [0]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

## Let's run the code!

In [0]:
for ep in range(n_epochs):
    for epi in range(n_episodes):
        # 랜덤하게 에피소드마다 n개의 클래스를 샘플링합니다.
        epi_classes = np.random.permutation(n_classes)[:n_way]
        support = np.zeros([n_way, n_shot, im_height, im_width], dtype=np.float32)
        query = np.zeros([n_way, n_query, im_height, im_width], dtype=np.float32)
        for i, epi_cls in enumerate(epi_classes):
            # 각 클래스에서 랜덤으로 shot + query 개수만큼의 이미지를 랜덤하게 샘플링합니다.
            selected = np.random.permutation(n_examples)[:n_shot + n_query]
            # 샘플링된 이미지 중 shot 개수만큼의 이미지가 support set이 됩니다.
            support[i] = train_dataset[epi_cls, selected[:n_shot]]
            # 샘플링된 이미지 중 남은 이미지는 query 이미지가 됩니다.
            query[i] = train_dataset[epi_cls, selected[n_shot:]]
        support = np.expand_dims(support, axis=-1)
        query = np.expand_dims(query, axis=-1)
        labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
        # session을 실행하여 훈련을 진행하고, loss와 accuracy를 출력합니다.
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={supports: support, queries: query, y:labels})
        if (epi+1) % 50 == 0:
            print('[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))

## Visualization

![대체 텍스트](https://drive.google.com/uc?id=1fx5bVyr-VTHjtW21bWGt-LeghmQ3Iaom)

In [0]:
epi_classes = np.random.permutation(n_classes)[:n_way]
support = np.zeros([n_way, n_shot, im_height, im_width], dtype=np.float32)
query = np.zeros([n_way, n_query, im_height, im_width], dtype=np.float32)
for i, epi_cls in enumerate(epi_classes):
  selected = np.random.permutation(n_examples)[:n_shot + n_query]
  support[i] = train_dataset[epi_cls, selected[:n_shot]]
  query[i] = train_dataset[epi_cls, selected[n_shot:]]
support = np.expand_dims(support, axis=-1)
query = np.expand_dims(query, axis=-1)
labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
proto_embs, query_embs, ac = sess.run([prototypes, emb_queries, acc], feed_dict={supports: support, queries: query, y:labels})

In [0]:
pca = PCA(n_components=2)
pca.fit(proto_embs)
proto = pca.transform(proto_embs)
query = pca.transform(query_embs)

In [0]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_title('Metric Space', fontsize = 20)
targets = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']
colors = ['r', 'g', 'b', 'y', 'c']
for target, color in zip(proto, colors):
    ax.scatter(target[0],
               target[1],
               c = color,
               s = 500,
               marker = '*')
labels = labels.flatten()
for target, label in zip(query, labels):
    ax.scatter(target[0],
               target[1],
               c = colors[int(label)],
               s = 25)

ax.legend(targets)


## Test

훈련한 네트워크를 가지고 
5 way 5 shot 테스트 데이터 셋에 대해 테스트해봅시다.

In [0]:
# 테스트 데이터 셋을 불러옵니다.
root_dir = './data/omniglot'
test_split_path = os.path.join(root_dir, 'split', 'test.txt')
with open(test_split_path, 'r') as test_split:
    test_classes = [line.rstrip() for line in test_split.readlines()]
n_test_classes = len(test_classes)
test_dataset = np.zeros([n_test_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(test_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        test_dataset[i, j] = im
print(test_dataset.shape)

In [0]:
# 에피소드 수, way, shot, query 수를 설정합니다.
n_test_episodes = 1000
n_test_way = 5
n_test_shot = 5
n_test_query = 15

In [0]:
print('Testing...')
avg_acc = 0.
for epi in range(n_test_episodes):
    epi_classes = np.random.permutation(n_test_classes)[:n_test_way]
    support = np.zeros([n_test_way, n_test_shot, im_height, im_width], dtype=np.float32)
    query = np.zeros([n_test_way, n_test_query, im_height, im_width], dtype=np.float32)
    for i, epi_cls in enumerate(epi_classes):
        selected = np.random.permutation(n_examples)[:n_test_shot + n_test_query]
        support[i] = test_dataset[epi_cls, selected[:n_test_shot]]
        query[i] = test_dataset[epi_cls, selected[n_test_shot:]]
    support = np.expand_dims(support, axis=-1)
    query = np.expand_dims(query, axis=-1)
    labels = np.tile(np.arange(n_test_way)[:, np.newaxis], (1, n_test_query)).astype(np.uint8)
    ls, ac = sess.run([ce_loss, acc], feed_dict={supports: support, queries: query, y:labels})
    avg_acc += ac
    if (epi+1) % 50 == 0:
        print('[test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_test_episodes, ls, ac))
avg_acc /= n_test_episodes
print('Average Test Accuracy: {:.5f}'.format(avg_acc))

## Report

1. 코드 빈칸 채우기. 정답을 캡쳐해서 레포트에 첨부해주세요(3점).

2. **(1)** 20 way 1 shot, **(2)** 20 way 5 shot을 훈련해보고 20 way 1 shot 문제에 대해 테스트한 정확도를 캡쳐하여 레포트에 첨부해주세요(1점). 두 결과를 비교하여 설명해주세요(1점). (3문장 이하로 적기)

3. 임베딩 벡터 간의 유사도를 계산하는 measure가 꼭 유클리디안 거리일 필요는 없습니다. cosine similarity(https://en.wikipedia.org/wiki/Cosine_similarity) 도 가능하겠죠? euclidean distance 대신 cosine similarity로 바꿔보세요.
5 way 5 shot 문제에 대해 네트워크를 훈련 시키고 구현한 함수와 훈련 정확도를 캡쳐해서 레포트에 첨부해주세요. (3점)

4. 임베딩 벡터 간의 유사도를 계산하는 measure가 고정된 함수일 필요는 없습니다. [CVPR18]RelationNet(http://openaccess.thecvf.com/content_cvpr_2018/papers/Sung_Learning_to_Compare_CVPR_2018_paper.pdf) 을 읽고, 

  a) 유사도를 측정하는 함수를 학습 할 수 있는 방안을 레포트에 설명해주세요. (2점) (3문장 이하로 적기, 구현 할 필요 없음. 하지만 구현을 하셔서 제출을 하면 코멘트 또는 피드백을 드리겠습니다.)
  
  b) 유사도를 측정하는 함수를 학습 할 수 있는 방안을 레포트에 설명 (0.5점, 3문장 이하), 구현하여 코드와 결과를 캡쳐하여 레포트에 제출 (1점)

제출: hayeon926@kaist.ac.kr